<a href="https://colab.research.google.com/github/KBVBrahmarao/datasciencecoursera/blob/main/Week3_IN_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentence Count using MapReduce (Sentence Count using MapReduce)
Write a MapReduce program to count the number of sentences in the input text.
A sentence is considered complete if it ends with:
.   ?   !

In [ ]:
#STEP 1 — Install Java
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-17-jdk-headless is already the newest version (17.0.17+10-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.


In [1]:
#Check Java:
!java -version

openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


In [2]:
#STEP 2 — Set Java Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-17-openjdk-amd64/bin"

In [3]:
#STEP 3 — Download and Extract Hadoop
!wget -q https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /content/hadoop

In [4]:
#STEP 4 — Set Hadoop Environment Variables
os.environ["HADOOP_HOME"] = "/content/hadoop"
os.environ["HADOOP_CONF_DIR"] = "/content/hadoop/etc/hadoop"
os.environ["PATH"] += ":/content/hadoop/bin:/content/hadoop/sbin"

In [5]:
#Extra STEP 1 — Set JAVA_HOME again
!export HADOOP_HOME=/content/Hadoop
!export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
!export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

In [10]:
#Extra Step 2 — Verify directories
!ls /content

hadoop	hadoop-3.3.6.tar.gz  sample_data


In [12]:
#ExtraSTEP 3 — Check Hadoop version again
%%bash
export HADOOP_HOME=/content/hadoop-3.3.6
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

echo "HADOOP_HOME=$HADOOP_HOME"
echo "JAVA_HOME=$JAVA_HOME"

HADOOP_HOME=/content/hadoop-3.3.6
JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64


In [13]:
#STEP 5 — Correct Hadoop Configuration Files - core-site.xml
%%writefile /content/hadoop/etc/hadoop/core-site.xml
<?xml version="1.0"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>file:///</value>
    </property>
</configuration>

Overwriting /content/hadoop/etc/hadoop/core-site.xml


In [14]:
#Step 6 - Create hdfs-site.xml (even though HDFS cannot run in Colab)
#Step 6 - Create hdfs-site.xml (even though HDFS cannot run in Colab)
%%writefile /content/hadoop/etc/hadoop/hdfs-site.xml
<?xml version="1.0"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>

Overwriting /content/hadoop/etc/hadoop/hdfs-site.xml


In [15]:
#STEP  7A — Verify that files are correct
!cat /content/hadoop/etc/hadoop/core-site.xml
!cat /content/hadoop/etc/hadoop/hdfs-site.xml

<?xml version="1.0"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>file:///</value>
    </property>
</configuration>
<?xml version="1.0"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>


In [16]:
#Step 7B - Verify the files were created
!ls /content/hadoop/etc/hadoop

capacity-scheduler.xml		  kms-log4j.properties
configuration.xsl		  kms-site.xml
container-executor.cfg		  log4j.properties
core-site.xml			  mapred-env.cmd
hadoop-env.cmd			  mapred-env.sh
hadoop-env.sh			  mapred-queues.xml.template
hadoop-metrics2.properties	  mapred-site.xml
hadoop-policy.xml		  shellprofile.d
hadoop-user-functions.sh.example  ssl-client.xml.example
hdfs-rbf-site.xml		  ssl-server.xml.example
hdfs-site.xml			  user_ec_policies.xml.template
httpfs-env.sh			  workers
httpfs-log4j.properties		  yarn-env.cmd
httpfs-site.xml			  yarn-env.sh
kms-acls.xml			  yarnservice-log4j.properties
kms-env.sh			  yarn-site.xml


In [17]:
#STEP 7C — Set correct config directory
%%bash
export HADOOP_HOME=/content/hadoop
export HADOOP_CONF_DIR=/content/hadoop/etc/hadoop
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

hadoop version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /content/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar


In [26]:
#STEP 7D — Delete old output
# If it's meeded, execute.
!rm -rf output

In [19]:
#NEXT IMPORTANT STEPs
#STEP 7 - Fix log4j warning (optional but recommended)
%%bash
cat > /content/hadoop/etc/hadoop/log4j.properties <<'EOF'
log4j.rootLogger=INFO, console
log4j.appender.console=org.apache.log4j.ConsoleAppender
log4j.appender.console.target=System.err
log4j.appender.console.layout=org.apache.log4j.PatternLayout
log4j.appender.console.layout.ConversionPattern=%d{ISO8601} %-5p %c: %m%n
EOF

In [20]:
#STEP 8 — Create Input File
%%bash
cat > input.txt <<EOF
Hello world! How are you?
This is Hadoop MapReduce.
Sentence counting is easy!
EOF

In [21]:
#Mapper Program (Python)
%%bash
cat > mapper.py <<EOF
#!/usr/bin/env python3
import sys

for line in sys.stdin:
    count = 0
    for ch in line:
        if ch in ['.', '?', '!']:
            count += 1
    if count > 0:
        print("sentence\t{}".format(count))
EOF

chmod +x mapper.py

In [22]:
#Reducer Program (Python)
%%bash
cat > reducer.py <<EOF
#!/usr/bin/env python3
import sys

total = 0
for line in sys.stdin:
    _, count = line.strip().split("\t")
    total += int(count)

print("Total Sentences =", total)
EOF

chmod +x reducer.py

In [23]:
#Run MapReduce Job
%%bash
$HADOOP_HOME/bin/hadoop jar \
$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-input input.txt \
-output sentence_output \
-mapper mapper.py \
-reducer reducer.py

2025-12-21 08:50:38,258 INFO  org.apache.hadoop.metrics2.impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-12-21 08:50:38,711 INFO  org.apache.hadoop.metrics2.impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-12-21 08:50:38,711 INFO  org.apache.hadoop.metrics2.impl.MetricsSystemImpl: JobTracker metrics system started
2025-12-21 08:50:38,755 WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-12-21 08:50:39,331 INFO  org.apache.hadoop.mapred.FileInputFormat: Total input files to process : 1
2025-12-21 08:50:39,396 INFO  org.apache.hadoop.mapreduce.JobSubmitter: number of splits:1
2025-12-21 08:50:40,009 INFO  org.apache.hadoop.mapreduce.JobSubmitter: Submitting tokens for job: job_local745744626_0001
2025-12-21 08:50:40,011 INFO  org.apache.hadoop.mapreduce.JobSubmitter: Executing with tokens: []
2025-12-21 08:50:40,324 INFO  org.apache.hadoop.mapreduce.Job: The url to track th

In [24]:
#View Output
%%bash
cat sentence_output/part-00000

Total Sentences = 4	
